In [3]:
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np

# Importing the data from sources

## Inequality measures Deininger and Squire

Not sure whether the good quality criteria in Forbes deals with the variable Quality or Q. In order to be confident of our data, we impose the two conditions. 
This leads us to 679 points which is very close to the 682 points of the paper. 

We also add 6.6 we the data is based on expenditure and not on income (on the same cell to be sure that it is not done twice. 

In [5]:
DS = pd.read_excel("Deininger_and Squire.XLS", usecols=["Code", "Quality", "Year", "Gini", "Q", 'Inc'])
DS = DS.query("Q == 'good' and Quality == 'accept'")
DS.drop_duplicates(["Code", "Year"], inplace=True)
DS['Gini'].astype(float, inplace=True)

DS.loc[DS['Inc']=='E', 'Gini'] += 6.6

DS = DS[["Code", "Year", "Gini"]]
DS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679 entries, 0 to 2631
Data columns (total 3 columns):
Code    679 non-null object
Year    679 non-null int64
Gini    679 non-null object
dtypes: int64(1), object(2)
memory usage: 21.2+ KB


## Educational data : Barro-Lee

We take the data directly from the BarroLee website. Cf list of name variable name.

In [6]:
male_educ = pd.read_csv("male_attainment_25_BarroLee.csv", usecols=["year", "WBcode", "yr_sch_sec"])
female_educ = pd.read_csv("female_attainment_25_BarroLee.csv", usecols=["year", "WBcode", "yr_sch_sec"])

In [7]:
print male_educ.info()
print female_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1898 entries, 0 to 1897
Data columns (total 3 columns):
year          1898 non-null int64
yr_sch_sec    1898 non-null float64
WBcode        1898 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 59.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1898 entries, 0 to 1897
Data columns (total 3 columns):
year          1898 non-null int64
yr_sch_sec    1898 non-null float64
WBcode        1898 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 59.3+ KB
None


## Income

We downloaded the data from data.worldbank. The name of the variable has changed from GNP to GNI we should investigate to decide whether or not the differences are big. 
We secretly hope that the data before 1995 hasn't been changed and consequently that the new name of the variable affects nothing for our study. 

inc = pd.read_excel("pwt56_forweb.xls", usecols=["Country", "Year", "RGDPCH"]).dropna(subset=["RGDPCH"])
country_dict = pd.read_csv("../data_source/country_code_list.csv")

country_dict.loc[:, 'country'] = country_dict['country'].apply(lambda x: x.upper())
country_dict.set_index('country', inplace=True)

inc.replace("CAPE VERDE IS.", "CABO VERDE", inplace=True)
inc.replace("CENTRAL AFR.R.", "CENTRAL AFRICAN REPUBLIC", inplace=True)
inc.replace("GUINEA-BISS", "GUINEA-BISSAU", inplace=True)

inc = inc.query("Country != 'REUNION'")

inc = inc.query("Country != 'ZAIRE'")

inc.replace("DOMINICAN REP.", "DOMINICAN REPUBLIC", inplace=True)
inc.replace("ST.KITTS&NEVIS", "SAINT KITTS AND NEVIS", inplace=True)
inc.replace("ST.LUCIA", "SAINT LUCIA", inplace=True)
inc.replace("TRINIDAD&TOBAGO", "TRINIDAD AND TOBAGO", inplace=True)
inc.replace("U.S.A.", "UNITED STATES OF AMERICA", inplace=True)
inc.replace("KOREA, REP.", "SOUTH KOREA", inplace=True)
inc.replace("SYRIA", "SYRIAN ARAB REPUBLIC", inplace=True)
inc.replace("UNITED ARAB E.", "UNITED ARAB EMIRATES", inplace=True)
inc.replace("YEMEN", "REPUBLIC OF YEMEN", inplace=True)

inc = inc.query("Country != 'GERMANY, EAST'")

inc.replace("GERMANY, WEST", "GERMANY", inplace=True)
inc.replace("U.K.", "UNITED KINGDOM", inplace=True)
inc.replace("U.S.S.R.", "RUSSIAN FEDERATION", inplace=True)
inc.replace("PAPUA N.GUINEA", "PAPUA NEW GUINEA", inplace=True)
inc.replace("SOLOMON IS.", "SOLOMON ISLANDS", inplace=True)
inc.replace("ST.VINCENT&GRE", "SAINT VINCENT AND THE GRENADINES", inplace=True)
inc.replace("WESTERN SAMOA", "SAMOA", inplace=True)

inc['code'] = inc['Country'].apply(lambda x: country_dict.loc[x])
inc.columns = ["Country", "year", "GDP_PC", "code"]
inc = inc[['code', 'year', 'GDP_PC']]

In [8]:
inc = pd.read_csv("GDP_PC_WB.csv", skiprows=4)
del inc["Country Name"], inc['Indicator Code'], inc['Indicator Name']
inc.set_index("Country Code", inplace=True)
inc = inc.stack().reset_index()
inc.columns = ["code", "year", "GNI_PC"]
print inc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10346 entries, 0 to 10345
Data columns (total 3 columns):
code      10346 non-null object
year      10346 non-null object
GNI_PC    10346 non-null float64
dtypes: float64(1), object(2)
memory usage: 323.3+ KB
None


## Price Level of Investment

Extracted from the Penn World Table 5.6 (as in Forbes). The name of the variable is PI (cf. documentation column number [15]). 

We now convert the Country name to the corresponding code using our good old dictionary. 

In [9]:
PPPI = pd.read_excel("pwt56_forweb.xls", usecols=["Country", "Year", "PI"]).dropna(subset=["PI"])
country_dict = pd.read_csv("../data_source/country_code_list.csv")

country_dict.loc[:, 'country'] = country_dict['country'].apply(lambda x: x.upper())
country_dict.set_index('country', inplace=True)

PPPI.replace("CAPE VERDE IS.", "CABO VERDE", inplace=True)
PPPI.replace("CENTRAL AFR.R.", "CENTRAL AFRICAN REPUBLIC", inplace=True)
PPPI.replace("GUINEA-BISS", "GUINEA-BISSAU", inplace=True)

PPPI = PPPI.query("Country != 'REUNION'")

PPPI = PPPI.query("Country != 'ZAIRE'")

PPPI.replace("DOMINICAN REP.", "DOMINICAN REPUBLIC", inplace=True)
PPPI.replace("ST.KITTS&NEVIS", "SAINT KITTS AND NEVIS", inplace=True)
PPPI.replace("ST.LUCIA", "SAINT LUCIA", inplace=True)
PPPI.replace("TRINIDAD&TOBAGO", "TRINIDAD AND TOBAGO", inplace=True)
PPPI.replace("U.S.A.", "UNITED STATES OF AMERICA", inplace=True)
PPPI.replace("KOREA, REP.", "SOUTH KOREA", inplace=True)
PPPI.replace("SYRIA", "SYRIAN ARAB REPUBLIC", inplace=True)
PPPI.replace("UNITED ARAB E.", "UNITED ARAB EMIRATES", inplace=True)
PPPI.replace("YEMEN", "REPUBLIC OF YEMEN", inplace=True)

PPPI = PPPI.query("Country != 'GERMANY, EAST'")

PPPI.replace("GERMANY, WEST", "GERMANY", inplace=True)
PPPI.replace("U.K.", "UNITED KINGDOM", inplace=True)
PPPI.replace("U.S.S.R.", "RUSSIAN FEDERATION", inplace=True)
PPPI.replace("PAPUA N.GUINEA", "PAPUA NEW GUINEA", inplace=True)
PPPI.replace("SOLOMON IS.", "SOLOMON ISLANDS", inplace=True)
PPPI.replace("ST.VINCENT&GRE", "SAINT VINCENT AND THE GRENADINES", inplace=True)
PPPI.replace("WESTERN SAMOA", "SAMOA", inplace=True)


PPPI['code'] = PPPI['Country'].apply(lambda x: country_dict.loc[x])
PPPI = PPPI[['code', 'Year', 'PI']]

/usr/local/lib/python2.7/dist-packages/pandas/core/index.py:1759: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return self._engine.get_loc(key)


In [10]:
PPPI.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4950 entries, 10 to 6533
Data columns (total 3 columns):
code    4950 non-null object
Year    4950 non-null int64
PI      4950 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 154.7+ KB


# Gathering the data in one frame

The first step is to remove rename the columns in preparation of the merging of the databases. 
We also remove all the duplicates in order to avoid the problems during the merging. 

### Drop duplicates and normalize the data from different sources

In [11]:
# rename the columns (in order to merge)
DS.columns = ["code", "year", "gini"]
PPPI.columns = ["code", "year", "PPPI"]
male_educ.columns = ["year", "sch_male", "code"]
female_educ.columns = ["year", "sch_female", "code"]

# drop duplicates in order to avoid problems during the merging
print "Income duplicates :", inc.duplicated(subset=['code', 'year']).sum()
print "Inequality duplicates :", DS.duplicated(subset=['code', 'year']).sum()
print "PPPI duplicates :", PPPI.duplicated(subset=['code', 'year']).sum()
print "Male educ duplicates :", male_educ.duplicated(subset=['code', 'year']).sum()
print "Female Educ duplicates :", female_educ.duplicated(subset=['code', 'year']).sum()

# normalize the data types
DS.loc[:, 'year'] = DS['year'].astype(int)
DS.loc[:, 'code'] = DS['code'].astype(str)
DS.loc[:, 'gini'] = DS['gini'].astype(float)

PPPI.loc[:, 'year'] = PPPI['year'].astype(int)
PPPI.loc[:, 'code'] = PPPI['code'].astype(str)
PPPI.loc[:, 'PPPI'] = PPPI['PPPI'].astype(float)

male_educ.loc[:, 'year'] = male_educ['year'].astype(int)
male_educ.loc[:, 'code'] = male_educ['code'].astype(str)
male_educ.loc[:, 'sch_male'] = male_educ['sch_male'].astype(float)

female_educ.loc[:, 'year'] = female_educ['year'].astype(int)
female_educ.loc[:, 'code'] = female_educ['code'].astype(str)
female_educ.loc[:, 'sch_female'] = female_educ['sch_female'].astype(float)

inc.loc[:, 'year'] = inc['year'].astype(int)
inc.loc[:, 'code'] = inc['code'].astype(str)
inc.loc[:, 'GNI_PC'] = inc['GNI_PC'].astype(float)

Income duplicates : 0
Inequality duplicates : 0
PPPI duplicates : 0
Male educ duplicates : 0
Female Educ duplicates : 0


We now have to make sure of the concordance country codes between bases...

We found some problematic codes in the income database : codes that do not appear in our good old country code dictionary. We see that the corresponding countries are not countries or negligeable countries so we can just drop them from the database. 

In [12]:
problematic_codes = list(set(inc.query("code not in " + str(country_dict['code'].values.tolist()))['code']))
country_code = pd.read_csv("GNI_per_capita_WB.csv", skiprows=4, usecols=['Country Name', 'Country Code'])
print country_code[country_code['Country Code'].apply(lambda x: x in problematic_codes)]
inc = inc.query("code in " + str(country_dict['code'].values.tolist()))

                                       Country Name Country Code
5                                        Arab World          ARB
34                   Central Europe and the Baltics          CEB
36                                  Channel Islands          CHI
46                           Caribbean small states          CSS
58            East Asia & Pacific (developing only)          EAP
59          East Asia & Pacific (all income levels)          EAS
60          Europe & Central Asia (developing only)          ECA
61        Europe & Central Asia (all income levels)          ECS
64                                        Euro area          EMU
69                                   European Union          EUU
70         Fragile and conflict affected situations          FCS
90                                      High income          HIC
93           Heavily indebted poor countries (HIPC)          HPC
119     Latin America & Caribbean (developing only)          LAC
125   Latin America & Car

In Dieninger and Squire some codes are not used anymore in the new nomenclature so we choose to change them to the new nomenclature. 

In [13]:
problematic_codes = list(set(DS.query("code not in " + str(country_dict['code'].values.tolist()))['code']))
country_code = pd.read_excel("Deininger_and Squire.XLS", usecols=["Code", "Country"])
print country_code[country_code['Code'].apply(lambda x: x in problematic_codes)].drop_duplicates()

DS.replace("BRS", "BLR", inplace=True)
DS.replace("CSR", "CZE", inplace=True)
DS.replace("KYR", "KGZ", inplace=True)
DS.replace("LAT", "LVA", inplace=True)
DS.replace("LIT", "LTU", inplace=True)
DS.replace("MLD", "MDA", inplace=True)
DS.replace("ROM", "ROU", inplace=True)
DS.replace("SLO", "SVK", inplace=True)
DS.replace("SVA", "SVN", inplace=True)
DS.replace("SUN", "RUS", inplace=True)
DS.replace("OAN", "TWN", inplace=True)

           Country Code
162        Belarus  BRS
580      Czech Rep  CSR
1407   Kyrgyz Rep.  KYR
1409        Latvia  LAT
1415     Lithuania  LIT
1541       Moldova  MLD
1914       Romania  ROM
1956        Slovak  SLO
1957   Slovak Rep.  SLO
1966      Slovenia  SVA
1981  Soviet Union  SUN
2151        Taiwan  OAN


In the educationnal data from Barro and Lee some codes are not used anymore in the new nomenclature so we choose to change them to the new nomenclature. 

In [14]:
problematic_codes = list(set(male_educ.query("code not in " + str(country_dict['code'].values.tolist()))['code']))
country_code = pd.read_csv("male_attainment_25_BarroLee.csv", usecols=["WBcode", "country"])
print country_code[country_code['WBcode'].apply(lambda x: x in problematic_codes)].drop_duplicates()

male_educ.replace("ROM", "MDA", inplace=True)
male_educ.replace("SER", "SRB", inplace=True)

                  country WBcode
1833  Republic of Moldova    ROM
1846               Serbia    SER


In [15]:
problematic_codes = list(set(female_educ.query("code not in " + str(country_dict['code'].values.tolist()))['code']))
country_code = pd.read_csv("female_attainment_25_BarroLee.csv", usecols=["WBcode", "country"])
print country_code[country_code['WBcode'].apply(lambda x: x in problematic_codes)].drop_duplicates()

male_educ.replace("ROM", "MDA", inplace=True)
male_educ.replace("SER", "SRB", inplace=True)

                  country WBcode
1833  Republic of Moldova    ROM
1846               Serbia    SER


### Merging the data

In [16]:
data_frame = pd.merge(DS, PPPI, on=['code', 'year'],how='outer')
data_frame = pd.merge(data_frame, male_educ, on=['code', 'year'], how='outer')
data_frame = pd.merge(data_frame, female_educ, on=['code', 'year'], how='outer')
data_frame = pd.merge(data_frame, inc, on=['code', 'year'], how='outer')
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10485 entries, 0 to 10484
Data columns (total 7 columns):
code          10485 non-null object
year          10485 non-null float64
gini          679 non-null float64
PPPI          4950 non-null float64
sch_male      1898 non-null float64
sch_female    1898 non-null float64
GNI_PC        8745 non-null float64
dtypes: float64(6), object(1)
memory usage: 655.3+ KB


### Taking the log of the GDP

In [17]:
data_frame["log(GNI_PC)"] = data_frame['GNI_PC'].apply(np.log)
del data_frame['GNI_PC']

### Resampling the data

Complicated task because it is done in a complex way in the paper. This particular point could be critized btw. 
One has to read the paper carefully to obtain information on how the resampling is done (note 8). The author choose not to use the mean of all values in the period but only the first values which is not the good way to do in my opinion but here we just redo the calculations. Because the data on inequality is sparse, she allows the values to be not only the first one but the closest from the first one in the considered period. 

In [18]:
data_frame.reset_index(inplace=True)
period = float(5)
tuples = zip(data_frame['code'],
             (((data_frame['year'].values - 1) // period) * period))
df_copy = data_frame.copy()
df_copy.index = pd.MultiIndex.from_tuples(tuples)
df_copy = df_copy.groupby(level=[0, 1]).last().dropna(how='all')
df_copy.index.names = ['code', 'year']

In [19]:
data_frame = df_copy
del df_copy
data_frame = data_frame[['gini', 'PPPI', 'sch_male', 'sch_female', 'log(GNI_PC)']]

In [20]:
data_frame.loc['CAN']

,gini,PPPI,sch_male,sch_female,log(GNI_PC)
year,,,,,
1945,NaN,93.36,1.93,2.28,NaN
1950,32.56,105.73,2.13,2.43,NaN
1955,32.04,107.67,2.34,2.59,7.738300
1960,31.61,103.34,2.54,2.62,7.835784
1965,32.30,107.06,2.80,2.79,8.322554
1970,31.62,116.46,3.28,3.10,8.543599
1975,31.00,93.33,3.68,3.45,9.316364
1980,32.81,74.53,3.81,3.57,9.546185
1985,27.56,78.84,4.03,3.93,9.966575


### Computing the growth column (of income and Gini)

In [21]:
data_frame.reset_index(inplace=True)
data_frame.sort_values(['code', 'year'], inplace=True)
data_frame.set_index(['code'], inplace=True)

In [22]:
new_frame = pd.DataFrame()
for country in set(data_frame.index.values):
    sel = data_frame.loc[country]
    if sel.shape != (6,):
        sel['growth'] = (sel.shift(-1)['log(GNI_PC)'] - sel['log(GNI_PC)']) / (sel.shift(-1)['year'] - sel['year'])
        # sel['gini_growth'] = (sel['gini'] - sel.shift(1)['gini']) / (sel.shift(-1)['year'] - sel['year'])
        new_frame = pd.concat([new_frame, sel])

In [23]:
new_frame.loc['CAN']

,year,gini,PPPI,sch_male,sch_female,log(GNI_PC),growth
code,,,,,,,
CAN,1945,NaN,93.36,1.93,2.28,NaN,NaN
CAN,1950,32.56,105.73,2.13,2.43,NaN,NaN
CAN,1955,32.04,107.67,2.34,2.59,7.738300,0.019497
CAN,1960,31.61,103.34,2.54,2.62,7.835784,0.097354
CAN,1965,32.30,107.06,2.80,2.79,8.322554,0.044209
CAN,1970,31.62,116.46,3.28,3.10,8.543599,0.154553
CAN,1975,31.00,93.33,3.68,3.45,9.316364,0.045964
CAN,1980,32.81,74.53,3.81,3.57,9.546185,0.084078
CAN,1985,27.56,78.84,4.03,3.93,9.966575,-0.014772


In [24]:
data_frame = new_frame.query("1985 >= year >= 1960").dropna(how='any')
del new_frame

### Removing countries that doesn't have at least 2 consecutives observations

In [25]:
new_frame = pd.DataFrame()
data_frame.reset_index(inplace=True)
for country in set(data_frame['code']):
    if len(data_frame.query("code == '" + str(country) + "'")) > 1:
        sel = data_frame.query("code == '" + str(country) + "'")
        sel = sel.loc[((sel.shift(-1).year - sel.year) == 5) | ((sel.year - sel.shift(1).year) == 5)]
        new_frame = pd.concat([new_frame, sel])
data_frame = new_frame
del new_frame
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 3 to 173
Data columns (total 8 columns):
code           181 non-null object
year           181 non-null float64
gini           181 non-null float64
PPPI           181 non-null float64
sch_male       181 non-null float64
sch_female     181 non-null float64
log(GNI_PC)    181 non-null float64
growth         181 non-null float64
dtypes: float64(7), object(1)
memory usage: 12.7+ KB


In [26]:
data_frame.sort_values(['code', 'year']).to_csv("forbes_dataset.csv", index=False)

# Comparing with the data presented in the paper

In [27]:
pd.set_option('precision', 3)
nb_per = len(set(data_frame.year))
resume = pd.DataFrame()
resume = pd.concat([resume, data_frame.groupby('year').mean()])
resume = pd.concat([resume, data_frame.groupby('year').std()])
resume = pd.concat([resume, data_frame.groupby('year').min()])
resume = pd.concat([resume, data_frame.groupby('year').max()])
del resume['code']
resume['categorize'] = (['mean'] * nb_per + ['std'] * nb_per + ['min'] * nb_per + ['max'] * nb_per) 
result = resume.reset_index().set_index(['categorize', 'year']).stack(level=0).unstack(level=0).swaplevel(0,1).sort_index()
print result[['mean', 'std', 'min', 'max']]

categorize          mean     std     min      max
            year                                 
PPPI        1960  77.385  21.762  40.750  119.160
            1965  67.948  18.928  41.150  107.060
            1970  85.604  24.082  36.450  139.580
            1975  95.598  30.612  35.330  187.260
            1980  66.842  23.813  31.860  162.850
            1985  79.609  32.627  27.910  136.140
gini        1960  38.587   8.197  24.300   55.500
            1965  40.824   9.558  25.100   57.700
            1970  40.665   9.464  23.300   61.940
            1975  38.794   9.021  24.900   63.180
            1980  38.545   8.070  23.420   61.760
            1985  39.211   8.374  24.530   59.600
growth      1960   0.058   0.038  -0.012    0.126
            1965   0.140   0.088  -0.061    0.320
            1970   0.120   0.063  -0.075    0.211
            1975   0.002   0.054  -0.084    0.135
            1980   0.096   0.071  -0.084    0.224
            1985   0.031   0.062  -0.146    0.131


In [28]:
print data_frame[['code', 'year', 'gini']].set_index(['code', 'year']).unstack(level=1)

       gini                                       
year   1960   1965    1970    1975    1980    1985
code                                              
AUS     NaN    NaN     NaN  39.330  37.580  41.720
BEL     NaN    NaN     NaN  28.250  26.222     NaN
BGD   37.31  34.20  36.000  35.170  36.000  35.450
BGR     NaN    NaN     NaN  25.010  23.420  24.530
BRA     NaN  57.61  61.940  57.780  61.760  59.600
CAN   31.61  32.30  31.620  31.000  32.810  27.560
CHL     NaN  45.64  46.000  53.210     NaN     NaN
CHN     NaN    NaN     NaN  32.000  31.400  34.600
CIV     NaN    NaN     NaN     NaN  47.810  43.490
COL     NaN  52.02  46.000  54.500     NaN     NaN
CRI     NaN    NaN  44.400  45.000  47.000  46.070
DEU     NaN  33.57  30.616  32.065  32.195     NaN
DNK     NaN    NaN     NaN  31.000  30.990  33.145
DOM     NaN    NaN     NaN  45.000  43.290     NaN
ESP     NaN    NaN  37.110  33.390  31.790  32.510
FIN     NaN  31.80  27.000  30.860  30.840  26.188
FRA   47.00  44.00  43.000  34.